<a href="https://colab.research.google.com/github/caohungphu/CS114.L21/blob/main/Colab/Predict_final_exam_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><b>CS114.L21 - MÁY HỌC - MACHINE LEARNING</b></h1>
<h1 align="center"><b>BÀI TẬP COLAB SỐ 03 (17/05/2021)</b></h1>
 
## **1. YÊU CẦU:**

- Data discovery


## **2. NHÓM THỰC HIỆN**
- Cao Hưng Phú - 19520214 - 19520214@gm.uit.edu.vn
- Nguyễn Thành Công - 19521294 - 19521294@gm.uit.edu.vn
- Trần Huỳnh Kỳ Anh - 19521216 - 19521216@gm.uit.edu.vn

## **3. LINK GITHUB**
- Link GitHub nhóm: https://github.com/caohungphu/CS114.L21/
- Link GitHub bài làm: [Predict_final_exam_score.ipynb](https://github.com/caohungphu/CS114.L21/blob/main/Colab/Predict_final_exam_score.ipynb)
- Link GitHub dataset: [Dataset](https://github.com/caohungphu/CS114.L21/tree/main/Colab/CSV/17_05_2021)

## **4. SOURCE CODE**

### **4.1. Nguồn tham khảo**

1. Thư viện pandas: https://pandas.pydata.org/docs/

### **4.2. Code**

#### **4.2.1. Thêm thư viện, đọc dữ liệu**

In [ ]:
# Thêm các thư viện cần thiết
import os
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
 
# Đường dẫn thư mục dataset
dir_dataset_folder = "/content/CS114.L21/Colab/CSV/17_05_2021/"
 
# Nếu không có đường dẫn thì clone từ GitHub về
if not os.path.exists(dir_dataset_folder):
    !git clone https://github.com/caohungphu/CS114.L21.git

In [ ]:
# Đọc dữ liệu từ file CSV
wecode_data = pd.read_csv("/content/CS114.L21/Colab/CSV/17_05_2021/wecode.csv")
score_training_data = pd.read_csv("/content/CS114.L21/Colab/CSV/17_05_2021/score_training.csv")

In [ ]:
# Lấy những cột cần thiết
wecode_data_new = wecode_data[['user_id', 'assignment_id', 'problem_id', 'is_final', 'status', 'pre_score', 'coefficient']]
score_training_data_new = score_training_data[['id', 'CK']]

#### **4.2.2. Tiền xử lí dữ liệu**

In [ ]:
# Hàm kiểm tra code đã solved chưa
def checkSolved(data):
  if (data['is_final'] == 1) and (data['pre_score'] == 10000):
    return 1
  return 0

# Thêm cột với điều kiện solved vào data
wecode_data_new['solved'] = wecode_data_new.apply(checkSolved, axis=1)

# Xóa những dòng duplicate của cùng 1 user giống problem_id nhưng khác assignment (Chắc do Practice)
# VD: user_id: 1868
wecode_data_new = wecode_data_new.drop_duplicates(subset=['user_id', 'problem_id', 'solved'])

In [ ]:
# Góm nhóm những người cùng user_id để tính các giá trị cần thiết mới
groupByUserID = wecode_data_new.groupby(by='user_id', sort = True)

wecode_data_new_header = pd.DataFrame({
    # Số assignment_tries
    'assignment_tries': groupByUserID['assignment_id'].nunique(),  
    # Số problem_tries 
    'problem_tries': groupByUserID['problem_id'].nunique(),
    # Số problem_solved 
    'problem_solved': groupByUserID['solved'].apply(lambda x: (x==1).sum())
})

# Thay index: user_id thành index mới và user_id thành một cột giá trị user_id
wecode_data_new_header = wecode_data_new_header.reset_index()

In [ ]:
# Đếm số assignment và problem
number_of_assignment = wecode_data_new['assignment_id'].nunique()
number_of_problem = wecode_data_new['problem_id'].nunique()
# Thêm cột solved_per_tries
wecode_data_new_header['solved_per_tries'] = wecode_data_new_header['problem_solved'] / wecode_data_new_header['problem_tries']
# Thêm cột problem_tries_per_problem
wecode_data_new_header['problem_tries_per_problems'] = wecode_data_new_header['problem_tries'] / number_of_problem
# Thêm cột assignment_tries_per_assignments
wecode_data_new_header['assignment_tries_per_assignments'] = wecode_data_new_header['assignment_tries'] / number_of_assignment
# P/s: 3 giá trị của cột trên gần = 1 là tốt

In [ ]:
wecode_data_new_header

,user_id,assignment_tries,problem_tries,problem_solved,solved_per_tries,problem_tries_per_problems,assignment_tries_per_assignments
0,1,1,2,1,0.500000,0.003155,0.003448
1,4,1,41,37,0.902439,0.064669,0.003448
2,5,4,5,3,0.600000,0.007886,0.013793
3,6,4,88,84,0.954545,0.138801,0.013793
4,7,1,2,0,0.000000,0.003155,0.003448
...,...,...,...,...,...,...,...
1154,1860,9,43,26,0.604651,0.067823,0.031034
1155,1862,2,6,4,0.666667,0.009464,0.006897
1156,1864,5,63,28,0.444444,0.099369,0.017241
1157,1865,3,17,10,0.588235,0.026814,0.010345


In [ ]:
# Thay header cho trùng với dữ liệu wecode
score_training_data_new.columns = ['user_id', 'CK']
# Thay đổi từ Object sang Numberic
score_training_data_new['CK'] = score_training_data_new['CK'].apply(pd.to_numeric, errors='coerce')

In [366]:
# Tạo 2 dataframe mới từ bộ dữ liệu điểm CK bị thiếu
score_train = score_training_data_new.dropna(how='any')
score_predict = score_training_data_new[score_training_data_new.isna().any(axis=1)]

# Thay giá trị index chạy từ 0
score_train = score_train.reset_index()
score_train = score_train.drop(columns=['index'])
score_predict = score_predict.reset_index()
score_predict = score_predict.drop(columns=['index'])

In [322]:
# Merge data với bộ dataset Wecode
training_data = pd.merge(score_train, wecode_data_new_header, how='inner', on='user_id')
predict_data = pd.merge(score_predict, wecode_data_new_header, how='inner', on='user_id')

#### **4.2.3. Thử nghiệm dữ liệu với SVR**

In [384]:
# Chuẩn bị model và xem kết quả training
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

X = np.array(training_data[['solved_per_tries', 'problem_tries_per_problems', 'assignment_tries_per_assignments']])
y = np.array(training_data['CK'])

clf = SVR().fit(X, y)
y_predict = clf.predict(X)

print("MSE:",mean_squared_error(y, y_predict))

MSE: 4.5655449907437635


In [385]:
# Dự đoán điểm cuối kỳ còn thiếu
predict_data['CK'] = clf.predict(predict_data[['solved_per_tries', 'problem_tries_per_problems', 'assignment_tries_per_assignments']])
predict_data

,user_id,CK,assignment_tries,problem_tries,problem_solved,solved_per_tries,problem_tries_per_problems,assignment_tries_per_assignments
0,51,6.113199,5,39,38,0.974359,0.061514,0.017241
1,240,6.784797,15,123,113,0.918699,0.194006,0.051724
2,391,6.613539,14,111,101,0.909910,0.175079,0.048276
3,513,7.034409,15,135,127,0.940741,0.212934,0.051724
4,586,6.940828,15,124,120,0.967742,0.195584,0.051724
5,705,6.918280,14,114,114,1.000000,0.179811,0.048276
6,772,6.416426,13,91,85,0.934066,0.143533,0.044828
7,774,6.658253,14,99,98,0.989899,0.156151,0.048276
8,776,6.675329,13,98,97,0.989796,0.154574,0.044828
9,792,6.641391,15,102,100,0.980392,0.160883,0.051724


In [386]:
# Làm tròn điểm số cuối kỳ
def x_round(x):
    x = round(x*2)/2
    return x

predict_data['CK'] = x_round(predict_data['CK'])
predict_data

,user_id,CK,assignment_tries,problem_tries,problem_solved,solved_per_tries,problem_tries_per_problems,assignment_tries_per_assignments
0,51,6.0,5,39,38,0.974359,0.061514,0.017241
1,240,7.0,15,123,113,0.918699,0.194006,0.051724
2,391,6.5,14,111,101,0.909910,0.175079,0.048276
3,513,7.0,15,135,127,0.940741,0.212934,0.051724
4,586,7.0,15,124,120,0.967742,0.195584,0.051724
5,705,7.0,14,114,114,1.000000,0.179811,0.048276
6,772,6.5,13,91,85,0.934066,0.143533,0.044828
7,774,6.5,14,99,98,0.989899,0.156151,0.048276
8,776,6.5,13,98,97,0.989796,0.154574,0.044828
9,792,6.5,15,102,100,0.980392,0.160883,0.051724


## **5. NHẬN XÉT VÀ TRẢ LỜI YÊU CẦU ĐỀ BÀI**

### **5.1. Nhận xét**


### **5.2. Trả lời yêu cầu đề bài**
